In [0]:
# From Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Load train and validation sets (both X and Y)
X_train_template = "/content/drive/My Drive/train/X_tensor_train_chunk_"
Y_train_template = "/content/drive/My Drive/train/Y_tensor_train_chunk_"

X_val_template = "/content/drive/My Drive/val/X_tensor_val_chunk_"
Y_val_template = "/content/drive/My Drive/val/Y_tensor_val_chunk_"

suffix = ".pkl"

minimum = 0
cut_1 = 33000
cut_2 = 40000
maximum = 100000
step = 1000

def partial_list(X_template,Y_template,left,right,step):
  return([ [X_template+str(chunk)+suffix, Y_template+str(chunk)+suffix] for chunk in range(left,right,step)])

# part 1: subject id 3 to 32811 (>= 0 and < 33000 in practice)
np_train_list_part1 = partial_list(X_train_template,Y_train_template,minimum,cut_1,step)
np_val_list_part1 = partial_list(X_val_template,Y_val_template,minimum,cut_1,step)

# part 2: subject id >= 40000
np_train_list_part2 = partial_list(X_train_template,Y_train_template,cut_2,maximum,step)
np_val_list_part2 = partial_list(X_val_template,Y_val_template,cut_2,maximum,step)

In [0]:
# p Try to release memory
from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session
import tensorflow

# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del classifier # this is from global space - change this as you need
    except:
        pass

    #print(gc.collect()) # if it's done something you should see a number being outputted

    # use the same config as you used to create the session
    config = tensorflow.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tensorflow.Session(config=config))

In [0]:
# Method that concatenates all the X's and all the Y's from a given list into
# an overall tuple
import numpy as np

def concat(np_list):
  
  x_final = np.load(np_list[0][0])[:,:,4:]
  y_final = np.load(np_list[0][1])[:,]
  
  for e in range(1,len(np_list)):
    x_final = np.concatenate((x_final, np.load(np_list[e][0])[:,:,4:]),axis=0)
    y_final = np.concatenate((y_final, np.load(np_list[e][1])[:,]),axis=0)
    
  print(x_final.shape)
  print(y_final.shape)
  
  return(x_final,y_final)

In [0]:
import pickle

x_train,y_train = concat(np_train_list_part1)

In [0]:
x_val,y_val = concat(np_val_list_part1)
x_val2,y_val2 = concat(np_val_list_part2)

In [0]:
x_train2,y_train2 = concat(np_train_list_part2)

In [0]:
x_train = np.concatenate((x_train,x_train2),axis=0)
#y_train = np.concatenate((y_train,y_train2),axis=0)
#print(x_train.shape)
#print(y_train.shape)

In [0]:
import keras
from keras.models import Sequential # needed to activate NN
from keras.layers import Dense, LSTM, Dropout # needed to add hidden layers (of three types in our case)
from keras.callbacks import ModelCheckpoint
import keras.optimizers
import keras.regularizers
import sys
!pip install keras_metrics # (you will need to install this package)
import keras_metrics

# Fixed parameters 
timesteps = 6 # duration of a slice
data_dim = 66 # number of features

def build_model(dimension_list, dropout_list,timesteps=6,data_dim=66, reg_param=0.001):
  
  model = Sequential()
  # ANU
  model.add(Dense(dimension_list[0], activation='relu',init='uniform', kernel_regularizer=keras.regularizers.l2(reg_param), input_shape=(timesteps, data_dim)))
  # ALDO
  #model.add(Dense(dimension_list[0], activation='relu',init='uniform', kernel_regularizer=keras.regularizers.l1(reg_param), input_shape=(timesteps, data_dim)))
  
  for i in range(1,len(dimension_list)-1):
    model.add(LSTM(dimension_list[i], return_sequences=True)) # returns a sequence of vectors of dimension "dim_i"
    model.add(Dropout(dropout_list[i-1]))
    
  model.add(LSTM(dimension_list[len(dimension_list)-1]))
  model.add(Dropout(dropout_list[len(dropout_list)-1]))
  model.add(Dense(1,activation='sigmoid'))
  
  return(model)

In [0]:
# Pick a model of your choice

dimension_choice = [512,512,512] # Harini had 512 twice, but here we are adding a dense layer first
dropout_choice = [0.3,0.3,0.3] # between 0.2 and 0.5

def model_name(dimension_choice,dropout_choice):
  
  dimension_choice_str = [str(e) for e in dimension_choice]
  dropout_choice_str = [str(int(e*100)) for e in dropout_choice]

  model_name = str(len(dimension_choice)) + "_dim_" + "_".join(dimension_choice_str) + "_drop_" + "_".join(dropout_choice_str)
  
  return(model_name)

model = build_model(dimension_choice, dropout_choice)
model_name = model_name(dimension_choice, dropout_choice)

proba_threshold = 0.5

# Compile it
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [keras_metrics.recall()])
                                                                           #,keras_metrics.precision(),auc_roc,'accuracy'])

In [0]:
# Save the model weights after each epoch if the validation loss decreased
data_path = '/content/drive/My Drive'
checkpointer = ModelCheckpoint(filepath=data_path + '/model_new_' + model_name +'_{epoch:02d}.hdf5', verbose=1, save_best_only=True)

In [0]:
# Determine class weights

from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
print(class_weights)

In [0]:
class_weight = {0: class_weights[0],
                1: class_weights[1]}

In [0]:
# Fit the model based on rebalanced dataset

num_epochs = 10
batch_size = 128
min_delta = 0.0001
patience = 3

model.fit(x_train, y_train, batch_size=batch_size, epochs=num_epochs, class_weight=class_weight, verbose=1, validation_data=(x_val,y_val), callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=min_delta, patience=patience, verbose=1, mode='auto'), checkpointer])

In [0]:
# Load test set

X_test_template = "/content/drive/My Drive/test/X_tensor_test_chunk_"
Y_test_template = "/content/drive/My Drive/test/Y_tensor_test_chunk_"

np_test_list_part1 = partial_list(X_test_template,Y_test_template,minimum,cut_1,step)
np_test_list_part2 = partial_list(X_test_template,Y_test_template,cut_2,maximum,step)

In [0]:
x_test1,y_test1 = concat(np_test_list_part1)

In [0]:
x_test2,y_test2 = concat(np_test_list_part2)
print(x_test.shape)
print(y_test.shape)

In [0]:
x_test = np.concatenate((x_test1,x_test2),axis=0)
y_test=np.concatenate((y_test1,y_test2),axis=0)

In [0]:
# Model evaluation
def recall(x_test,y_test,model,batch_size):
  evaluation = model.evaluate(x_test,y_test,batch_size=batch_size)
  recall = evaluation[1]
  return(recall)

print(recall(x_test,y_test,model,batch_size))